来源：https://github.com/tensorflow/tensorflow/blob/r1.13/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb

In [1]:
%config IPCompleter.greedy=True

# 生成对抗网络(GANs)

生成对抗网络，是估计生成模型的框架。

敌对的两个模型同时训练的过程:
- 一个__生成器模型__,它负责生成数据(图像)
- 一个__鉴别器模型__,它负责评估一个图像的概率。从训练数据(图像是真实的),或者是由生成器(图片是假的)。

在训练过程中，生成器在生成图像方面变得越来越好，直到识别器不再能够区分真假图像

In [2]:
import tensorflow as tf 

In [3]:
tf.enable_eager_execution()

In [4]:
import time
import os
import numpy as np 
import glob
import imageio
import matplotlib.pyplot as plt
import PIL

## 加载数据

In [8]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

In [5]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [6]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## 对抗模型

### 生成器模型
该生成器负责创建令人信服的图像，这些图像足够好，足以骗过鉴别器。

该生成器的网络结构由`Conv2dTranspose`(反卷积)层组成。

我们从一个完全连接的图层开始，对图像进行两次采样，以达到理想的图像大小28x28x1。

增加宽度和高度，并减少深度，因为我们通过网络层移动。

除了最后一层使用tanh激活外，我们对每一层都使用泄漏ReLU激活

In [10]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
      
    model.add(tf.keras.layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size
    
    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)  
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)    
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)
  
    return model

### 鉴别器模型
__鉴别器__负责鉴别真假图像。类似于一个常规的基于cnn的图像分类器

__卷积输出边长公式__:
    `valid边长 = (输入边长-核边长+1)/步长`，`same边长 = (输入边长)/步长`

__卷积输出通道__:__卷积核个数相同__

In [11]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
      
    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))
       
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
     
    return model

In [12]:
generator = make_generator_model()
discriminator = make_discriminator_model()

## 损失

### 生成器损失
__生成器损失__是生成的图像和一组图像的s形交叉熵损失，因为__生成器__试图__生成__与真实图像相似的__伪图像__

In [13]:
def generator_loss(generated_output):
    return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output), generated_output)

### 鉴别器损失
__鉴别器损失__函数接受两个输入：__真实图像__和__生成的图像__
- 计算真实图像与一组1的交叉熵损失
- 计算伪  图像与一组0的交叉熵损失，伪图像是由生成器生成的图像
- 计算2组交叉熵的和

In [14]:
def discriminator_loss(real_output, generated_output):
    
    real_loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=tf.ones_like(real_output),
        logits=real_output)

    generated_loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=tf.zeros_like(generated_output), 
        logits=generated_output)

    total_loss = real_loss + generated_loss
    return total_loss

## 优化器

In [15]:
generator_optimizer = tf.train.AdamOptimizer(1e-4)
discriminator_optimizer = tf.train.AdamOptimizer(1e-4)

创建检查点

In [16]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## 开始训练

定义训练参数

In [17]:
noise_dim = 100
EPOCHS = 50
num_examples_to_generate = 16

random_vector_for_generation  = tf.random_normal([num_examples_to_generate, noise_dim])

__定义训练方法__

步骤：
- 给出一个__随机向量__作为输入
- 处理后输出一个看起来像手写数字的__生成图像__。
- 然后向鉴别器显示__真实MNIST图像__以及__生成的图像__。
- 接下来，我们计算了__生成器__和__鉴别器__的损失。
- 然后，我们计算了__生成器变量__和__鉴别器变量__的__梯度__。

In [18]:
def train_step(images):
   #  随机的噪声向量
      noise = tf.random_normal([BATCH_SIZE, noise_dim])
      
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
#         生成手写数字图像
        generated_images = generator(noise, training=True)
    
#         向鉴别器输入真实MNIST图像  
        real_output = discriminator(images, training=True)
#         向鉴别器输入生成的数字图像
        generated_output = discriminator(generated_images, training=True)
         
#         计算生成器的损失
        gen_loss = generator_loss(generated_output)
#         计算鉴别器的损失
        disc_loss = discriminator_loss(real_output, generated_output)
        
#       生成器梯度计算
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.variables)
#       鉴别器梯度计算
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.variables)
      
#       生成优化器应用梯度结果于生成器变量更新
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.variables))
#       鉴别优化器应用梯度结果于生成器变量更新
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.variables))

这个方法在单个colab上训练将花费大约30秒每轮，

急于执行可能比执行等价的图慢，因为它不能从图上的整个程序优化中获益，而且还会导致解释Python代码的开销。

通过使用`tf.contrib.eager.defun`来创建图形函数，我们获得了~20秒/epoch的性能提升(从~50秒/epoch下降到~30秒/epoch)。

通过这种方式，我们可以同时获得快速执行(更容易调试)和图形模式(更好的性能)的最佳效果。

In [19]:
train_step = tf.contrib.eager.defun(train_step)

In [27]:
def train(dataset, epochs):  
    for epoch in range(epochs):
        start = time.time()
    
        for images in dataset:
            train_step(images)

        display.clear_output(wait=True)
        generate_and_save_images(generator,
                               epoch + 1,
                               random_vector_for_generation)
    
    # saving (checkpoint) the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
    
        print ('Time taken for epoch {} is {} sec'.format(epoch + 1,
                                                      time.time()-start))
  # generating after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,        
                           epochs,                    
                           random_vector_for_generation)




保存生成的图像

In [21]:
def generate_and_save_images(model, epoch, test_input):
  # make sure the training parameter is set to False because we
  # don't want to train the batchnorm layer when doing inference.
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
        
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## 训练GANs

我们将调用上面定义的train()方法来同时训练生成器和鉴别器。注意，训练是很棘手的。

重要的是，生成器和鉴别器不能互相压倒对方(例如，它们的训练速度相似)。

在训练开始时，生成的图像看起来像随机噪声。
随着训练的进展，您可以看到生成的数字看起来越来越真实。
经过50个轮，他们看起来非常像MNIST数字。

In [29]:
%%time
train(train_dataset, EPOCHS)

SystemError: <class 'tensorflow.python.eager.backprop._MockOp'> returned a result with an error set

载入最后的检查点

In [23]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## 生成图像

In [24]:
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [25]:
display_image(EPOCHS)

FileNotFoundError: [Errno 2] No such file or directory: 'image_at_epoch_0050.png'

将保存的生成图像生成gif

In [ ]:
with imageio.get_writer('dcgan.gif', mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
# this is a hack to display the gif inside the notebook
os.system('cp dcgan.gif dcgan.gif.png')

In [ ]:
display.Image(filename="dcgan.gif.png")

In [ ]:
#from google.colab import files
#files.download('dcgan.gif')

# 以下属于功能测试

In [ ]:
dense = tf.keras.layers.Dense(7*7*256,input_shape=(100,))

In [ ]:
dense_out=dense(noise)
dense_out.shape

In [ ]:
dense_out[0]

In [ ]:
bn = tf.keras.layers.BatchNormalization()
# bn(dense_out)

In [ ]:
reshape = tf.keras.layers.Reshape((7, 7, 256))

In [ ]:
reshape_out = reshape(dense_out)
reshape_out.shape

In [ ]:
cts = tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same')

In [ ]:
cts_out = cts(reshape_out)
cts_out.shape

In [30]:
image_shape=(224,224)
image_shape = image_shape+(3,)
image_shape

(224, 224, 3)